### Script Purpose
- Get BMF file 2014-16 from NCCS.
- Use only A confidence level records.
- Drop organizations that changed their NTEE codes between 14-16.

In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import math
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1608['YR_MO']=1608
df_bmf_1512['YR_MO']=1512
df_bmf_1412['YR_MO']=1412

In [3]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16.loc[df_bmf_14_16[df_bmf_14_16.nteeConf=='a'].index, 'nteeConf']='A' # Correct erroneous records 'a'.
df_bmf_14_16_confA=df_bmf_14_16[df_bmf_14_16.nteeConf=='A'] # Select A confidence level records.

In [4]:
df_bmf_14_16.groupby('nteeConf').count()['EIN']/len(df_bmf_14_16)

nteeConf
A    0.561196
B    0.373232
C    0.065553
Name: EIN, dtype: float64

In [5]:
df_bmf_14_16_confA.sample(5)

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
3177192,133511640,36061.0,B90,Y,N,AMERICAN REGIONS MATHEMATICS LEAGUE INC,241 CENTRAL PARK WEST,NEW YORK,NY,10024,...,N,201412.0,147132.0,300506.0,bmf1512c3,Z,2014.0,0,611710.0,1512
3728044,364651294,54049.0,G25,N,N,STATE OF WEST VIRGINIA WEST VIRGINIA UNIVERSITY,586 MODS RUN RD,MANNINGTON,WV,26582,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813212.0,1512
2814054,742573896,48051.0,J40,Y,Y,TEXAS EXTENSION EDUCATION ASSOCIATION,PO BOX 125,SNOOK,TX,77878,...,Y,201312.0,NaN,NaN,submaster990n,Z,NaN,1,813930.0,1412
2434927,460518624,48157.0,P40,Y,Y,MOMS CLUB,2823 COTTON STOCK DR,SUGAR LAND,TX,77479,...,Y,201306.0,NaN,NaN,submaster990n,Z,NaN,1,624100.0,1412
3707820,363322126,46081.0,X21,N,N,ST JOHNS EPISCOPAL CHURCH ENDOWMENT TRUST,PO BOX 130,DEADWOOD,SD,57732,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813110.0,1512


**Drop organizations that changed their NTEE codes between 14-16.**

In [6]:
import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [7]:
dview.execute('import pandas as pd')
dview['df_bmf_14_16_confA']=df_bmf_14_16_confA
dview['df_bmf_14_16_confA_ntee_chg']=pd.DataFrame()

In [8]:
# Orgs changed their NTEE codes.
@dview.parallel(block=True)
def ntee_chg(ein):
    global df_bmf_14_16_confA, df_bmf_14_16_confA_ntee_chg
    ntee1_list=df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]['NTEE1']
    if len(set(ntee1_list))!=1:
        df_bmf_14_16_confA_ntee_chg=pd.concat([df_bmf_14_16_confA_ntee_chg, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]]
                                             )

In [9]:
t=ntee_chg.map(df_bmf_14_16_confA.EIN.unique())

In [15]:
df_bmf_14_16_confA_ntee_chg=pd.concat(dview.gather('df_bmf_14_16_confA_ntee_chg'), ignore_index=False)
df_bmf_14_16_confA_ntee_chg[['EIN', 'YR_MO', 'NTEE1']].head(5)

,EIN,YR_MO,NTEE1
65,10063430,1608,C
1584662,10063430,1412,Y
3081824,10063430,1512,Y
113,10132625,1608,U
1584710,10132625,1412,Y


In [16]:
dview['ein_drop_list']=df_bmf_14_16_confA_ntee_chg.EIN.unique()
dview['df_bmf_14_16_confA_chg_drop']=pd.DataFrame()

In [17]:
@dview.parallel(block=True)
def func_ntee_chg_drop(ein):
    global df_bmf_14_16_confA_chg_drop, df_bmf_14_16_confA
    if ein not in ein_drop_list:
        df_bmf_14_16_confA_chg_drop=pd.concat([df_bmf_14_16_confA_chg_drop, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]
                                              ])

In [18]:
t=func_ntee_chg_drop.map(df_bmf_14_16_confA.EIN.unique())

In [19]:
df_bmf_14_16_confA_chg_drop=pd.concat(dview.gather('df_bmf_14_16_confA_chg_drop')).sort_index()

In [24]:
len(df_bmf_14_16_confA_chg_drop)

2548479

In [23]:
df_to_write=df_bmf_14_16_confA_chg_drop
num_file=4
file_path_name='../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/df_bmf_14_16_confA_chg_drop.pkl.gz'
for index in range(0, df_to_write.iloc[-1].name+1, math.ceil(df_to_write.iloc[-1].name/num_file)):
    df_temp=df_to_write.loc[index:index+math.ceil(df_to_write.iloc[-1].name/num_file)-1]
    df_temp.to_pickle(file_path_name+'-'+str(df_temp.iloc[0].name)+'-'+str(df_temp.iloc[-1].name), compression='gzip')

In [26]:
# Test reading file.
import os
file_list=os.listdir('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/')
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, 
                       pd.read_pickle('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'+file, compression='gzip')])
len(df_test)

2548479

**Done with data acquisition, see how many changed.**

In [20]:
df_bmf_14_16_confA_chg_drop=pd.DataFrame()
for file in os.listdir('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'):
    df_bmf_14_16_confA_chg_drop=pd.concat([df_bmf_14_16_confA_chg_drop,
                                           pd.read_pickle('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'+file, compression='gzip')
                                          ])

In [23]:
[len(df_bmf_14_16_confA_chg_drop.EIN.unique()), 
 len(df_bmf_14_16_confA), 
 len(df_bmf_14_16_confA_chg_drop.EIN.unique())/len(df_bmf_14_16_confA)
 1-len(df_bmf_14_16_confA_chg_drop.EIN.unique())/len(df_bmf_14_16_confA)
]

911423

In [44]:
len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(101045, 960315)

In [45]:
len(df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates()), len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(977566, 960315)

In [40]:
df_bmf_14_16_confA_rm_chg=df_bmf_14_16_confA.loc[df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates(keep=False).index]

In [43]:
len(df_bmf_14_16_confA_rm_chg[['EIN', 'NTEE1']].drop_duplicates(keep=False)), len(df_bmf_14_16_confA_rm_chg)

(199801, 214947)

In [14]:
len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf']].drop_duplicates()), len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf', 'YR_MO']].drop_duplicates())

(1792488, 4633436)